In [37]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.messages import HumanMessage
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_classic import hub

In [2]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대한 짧은 설명을 해주세요.'
)
model = ChatOpenAI(model='gpt-5-nano', temperature=0)

In [3]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke('더블딥'))

더블딥(double-dip)은 경제가 한 차례 회복 국면에 접했다가 다시 경기 침체로 되돌아오는 현상을 말합니다. 보통 회복 국면이 끝나고도 소비와 투자 위축이 이어지며 성장률이 다시 음(-)으로 내려갈 때 발생합니다. 주로 금융위기 이후의 정책 효과 한계나 수요 둔화가 원인으로 꼽히고, V자형 반등과 달리 재침체의 가능성이 포함된다는 점이 특징입니다. 
혹시 다른 맥락(예: 딥러닝의 특정 용어 등)을 원하시면 알려주세요.


In [6]:
print(await chain.abatch([{'topic':'더블딥'}, {'topic':'인플레이션'}]))

['더블딥은 한 차례 회복 국면을 보이다가 다시 침체에 빠지는 경제 현상을 말합니다. 보통 실질 GDP가 두 분기 연속 감소하거나, 회복이 짧고 곧바로 재침체가 나타나는 상황을 뜻합니다. 원인으로는 수요 감소, 고용 악화, 금리 정책의 제약, 국제적 충격 등이 있습니다. 이 현상은 실업과 투자 축소를 야기해 정책 대응을 어렵게 만들 수 있습니다.', '인플레이션은 일반 물가가 지속적으로 상승하고 돈의 구매력이 떨어지는 현상입니다.  \n- 원인: 수요가 공급보다 빨리 늘어나거나 생산비가 오르는 비용 상승, 또는 통화 공급 증가 등.  \n- 측정: 물가 지수(CPI 등)로 측정합니다.  \n- 영향: 가계의 실질 구매력 감소, 금리와 임금 등에 영향, 자산가격 변동 등.  \n- 정책: 중앙은행이 목표 물가를 설정하고 금리 조정 등 통화정책으로 관리하려고 합니다.']


In [5]:
for token in chain.stream({'topic':'더블딥'}):
    print(token, end='', flush=True)

더블딥은 경제에서 한 차례 침체가 끝난 뒤 짧은 회복을 보이다가 다시 침체로 빠지는 현상을 말합니다. 보통 실질 GDP가 두 차례 연속으로 감소하는 상황으로 정의되며, 수요 부진이나 금융 충격, 정책 시차 등이 원인으로 꼽힙니다.

In [7]:
eng_prompt = ChatPromptTemplate.from_template('이 대답을 영어로 번역해주세요.: {answer}')

In [8]:
composed_chain = (
    {'answer':chain}
    | eng_prompt
    | model
    | StrOutputParser()
)

for token in composed_chain.stream({'topic':'인플레이션'}):
    print(token, end='', flush=True)

- Definition: A phenomenon where the general price level rises persistently and the purchasing power of money declines.

- Measurement/Causes: Measured by the inflation rate (e.g., CPI, etc.); main causes include demand-pull inflation, cost-push inflation, and inflation expectations.

- Effects: Higher cost of living reduces households' purchasing power, increases uncertainty, and leads to redistribution of income and wealth.

- Policy responses: Central banks adjust interest rates and manage the money supply; if necessary, fiscal policy can be used.

In [10]:
kor_chain = (
    ChatPromptTemplate.from_template('{topic}에 대해 짧은 설명을 해주세요.')
    | model
    | StrOutputParser()
)

eng_chain = (
    ChatPromptTemplate.from_template('{topic}에 대해 짧게 영어로 설명을 해주세요.')
    | model
    | StrOutputParser()
)

parallel_chain = RunnableParallel(kor=kor_chain, eng=eng_chain)
result = await parallel_chain.ainvoke({'topic':'인플레이션'})

print(result['kor'])
print('--'*30)
print(result['eng'])

인플레이션은 일정 기간 동안 재화와 서비스의 물가가 전반적으로 올라가 돈의 구매력이 떨어지는 현상입니다. 주된 원인은 수요가 공급을 넘치거나 생산비용 상승, 통화 공급 확대 등이 있습니다. 이를 측정하는 대표 지표로 CPI가 많이 쓰이며, 중앙은행은 물가 안정을 위해 금리 등을 조절합니다. 물가가 오르면 생활비가 증가하고 저축의 실질 가치가 감소하는 등의 영향이 있습니다.
------------------------------------------------------------
Inflation is the rate at which prices for goods and services rise over time, so the money you have buys less. It’s usually measured by price indices like the Consumer Price Index (CPI), and governments/central banks try to keep it low and stable.


In [12]:
template = PromptTemplate.from_template('주제 {topic}에 대해 금융 관련 짧은 조언을 해주세요.')
print(template.invoke({'topic':'투자'}))
print(template.invoke('투자'))

text='주제 투자에 대해 금융 관련 짧은 조언을 해주세요.'
text='주제 투자에 대해 금융 관련 짧은 조언을 해주세요.'


In [13]:
chain = (
    template
    | model
    | StrOutputParser()
)
print(chain.invoke({'topic':'더블딥'}))

다음은 더블딛(Double Dip) 상황에 대비한 짧은 금융 조언입니다.

- 더블딥이란 경기 회복이 잠시 멈추고 다시 침체로 돌아갈 위험을 말합니다. 불확실성에 대비해 현금 흐름과 자산구성을 점검하세요.
- 현금 비중과 고품질 자산 비중을 우선 확보하세요. 긴급자금과 안정적인 자산으로 변동성 충격을 흡수하는 것이 중요합니다.
- 분산 투자에 집중하고 방어적인 구성으로 재편해 보세요. 주식은 글로벌 분산과 함께 방어적 섹터를 포함하고, 채권은 단기/고품질 위주로 준비합니다.
- 레버리지 축소와 비용 관리에 신경 쓰세요. 수수료를 낮추고 불필요한 차입 위험은 피하는 것이 장기적으로 도움이 됩니다.
- 손실 한도를 설정하고 정기적으로 포트폴리오를 리밸런싱하세요. 필요하면 위험 관리 차원에서 헤지나 안전자산 비중 조정을 고려합니다.

원하시면 귀하의 투자 목표와 기간에 맞춘 간단한 시나리오를 함께 구성해 드릴게요.


In [16]:
template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('user', '주제 {topic}에 대해 금융 관련 주언을 해주세요.')]
)
print(template.invoke({'topic':'주식'}))
print(template.invoke('주식'))

messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 주식에 대해 금융 관련 주언을 해주세요.', additional_kwargs={}, response_metadata={})]
messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 주식에 대해 금융 관련 주언을 해주세요.', additional_kwargs={}, response_metadata={})]


In [22]:
template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('placeholder', '{msgs}')]
)
print(template.invoke({'msgs':[HumanMessage(content='안녕하세요.')]}))

messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요.', additional_kwargs={}, response_metadata={})]


In [23]:
examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자
""",
    } ,
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
""",
    },
]

In [26]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})

In [34]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=embeddings,
    vectorstore_cls=Chroma,
    k=1
)

question = '부동산 투자의 장점은 무엇인가?'
selected_examples = example_selector.select_examples({'question':question})

for example in selected_examples:
    for k, v in reversed(example.items()):
        print(f'{k}: {v}')

question: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
answer: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권



In [35]:
example_prompt = PromptTemplate(
    input_variables=['question', 'answer'],
    template='질문: {question}\n답변: {answer}'
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix='다음은 금융 관련 질문과 답변의 예입니다.:',
    suffix='질문: {question}\n답변: ',
    input_variables=['question']
)

In [36]:
chain = prompt | model
response = chain.invoke({'question':'부동산 투자의 장점은 무엇인가?'})
print(response.content)

답변:
- 안정적 현금흐름: 임대료 수입으로 정기적인 현금흐름을 창출할 수 있습니다.
- 자본 상승 여력: 장기적으로 부동산 가치가 상승할 가능성이 있습니다.
- 인플레이션 헤지: 임대료와 자산 가치가 물가 상승에 맞춰 조정되는 경향이 있어 인플레이션에 대응할 수 있습니다.
- 레버리지 활용 가능: 대출을 이용해 적은 초기 자본으로 큰 자산을 소유하고 수익을 확대할 수 있습니다(다만 위험도 증가).
- 포트폴리오 다각화: 주식·채권 외의 실물자산으로 포트폴리오의 위험 분산에 도움됩니다.
- 실물자산의 가치 창출 기회: 관리·개선이 수익성 증가로 이어질 수 있습니다.
- 세금 혜택 가능성: 지역에 따라 감가상각, 이자 비용 공제 등 혜택이 있을 수 있습니다.

참고: 구체적인 지역·세법에 따라 혜택과 위험이 달라지므로, 관심 지역이나 자금 규모를 알려주시면 해당 상황에 맞춘 장점과 예시를 더 자세히 정리해 드리겠습니다.


In [39]:
prompt = hub.pull('hardkothari/prompt-maker')
print(prompt)

input_variables=['lazy_prompt', 'task'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hardkothari', 'lc_hub_repo': 'prompt-maker', 'lc_hub_commit_hash': 'c5db8eeefa7be4862a9599b759608dd10ee53f53910838f69abb5ab31c257c2d'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert Prompt Writer for Large Language Models.\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lazy_prompt', 'task'], input_types={}, partial_variables={}, template='Your goal is to improve the prompt given below for {task} :\n--------------------\n\nPrompt: {lazy_prompt}\n\n--------------------\n\nHere are several tips on writing great prompts:\n\n-------\n\nStart the prompt by stating that it is an expert in the subject.\n\nPut instructions at the beginning of the prompt and use ### or to separate the instruction and context \n\nBe specific, descriptive and as deta